In [76]:
# importing necessary libraries
import pandas as pd

In [77]:
fp = r"C:\Users\Tino\Desktop\Python\Steam review scraping\reviews.csv"
df = pd.read_csv(fp, sep=";", index_col=0)
df.head()

,Found helpful,Recommend,Hours on record,Date posted,Review text
0,1839,True,46.3,"1 November, 2022","“Don't Be Sorry, Be Better.”"
1,4301,True,42.9,"8 November, 2022",Bring God Of War Ragnarok on PC
2,1860,True,18.2,"14 January, 2022",It's rare to see this much polish (on a consol...
3,7225,True,36.4,"17 January, 2022","No additional account, no unnecessary launcher..."
4,2340,True,11.3,"22 December, 2022",hi


In [78]:
# Checking if there is empty rows.
df.isna().sum()

Found helpful        0
Recommend            0
Hours on record      0
Date posted          0
Review text        154
dtype: int64

In [79]:
# Droppping empty review rows and columns that wont be needed for NLP.
df = df.dropna()[["Recommend", "Review text"]]
df.head()

,Recommend,Review text
0,True,"“Don't Be Sorry, Be Better.”"
1,True,Bring God Of War Ragnarok on PC
2,True,It's rare to see this much polish (on a consol...
3,True,"No additional account, no unnecessary launcher..."
4,True,hi


In [80]:
# There might be some cleaning that needs to be done in the data, lets check what kind of reviews the short reviews are.
# Lets create a boolean mask for the dataframe which tells us if the lenght of the review is under 5 chars long.
mask = df["Review text"].str.len() < 5
# Lets review the what kind of unique reviews the "review text" column contains.
df.loc[mask]["Review text"].unique()

array(['hi', 'BOY', 'boy', ' BOY', 'Boy.', 'BOY!', '👍', 'boy.', 'boi',
       'BOI', ' ᠌', 'Boi.', 'Boy', ':3', 'Boi!', '`', 'OwO', 'good',
       'yes', 'BOY.', 'ok', 'Yes', 'goty', '^_^', 'Boi', 'Boy!', 'GOTY',
       'yes!', 'fun', 'boi.', 'Buoy', 'angy', 'ong', 'game', '.', 'e',
       'Yes.', 'Good', '1337', 'bOY', 'WoaW', 'gg', '++++', 'boii', 'hjj',
       '!', 'ggo', 'gud', 'nice', 'HI', 'GOOD', 'pog', 'yup', 'Gud',
       'boat', 'Ω', 'WOW', 'øks', 'bald', 'epic', 'Fire', 'real', 'peak',
       'bro', 'l', 'm', 'god', 'GAME', 'jeng', 'kill', '9/10', 'Nice',
       'boy!', 'Hmm', 'ies', '3>', ':)', 'top', '1+', 'GOAT', 'dem',
       'yea', 'Epic', 'BOY+', '👍👍👍', '<3', 'gda', 'ye', 'SEX', 'mhm',
       'NKJH', 'لا', 'w', ':D', 'Sure', 'Bald', 'swag', 'voi¡', 'GUD',
       '5*', 'best', 'yes.', '❤', 'BOI.', ';)', '...', 'YOB', 'WOW!',
       ' boy', '+', 'BOI!', 'yeah', '5/5', 'ujgb', 'GOD', 'fye', 'ggz',
       'Fr', 'A1', 'God.', 'ᛒᛟᛁ', 'Gid.', 'gud!', 'ygh', 'yep', '[',
      

In [93]:
# Looks like we wont be missing too much valuable training data if we drop the reviews that are shorter than 5 lenght wise.
# Using the mask we created to drop the unwanted values and updating the indices.
df = df.drop(df.loc[mask].index).reset_index(drop=True)
df

,Recommend,Review text
0,True,"“Don't Be Sorry, Be Better.”"
1,True,Bring God Of War Ragnarok on PC
2,True,It's rare to see this much polish (on a consol...
3,True,"No additional account, no unnecessary launcher..."
4,True,god of war is one of those games i wish i coul...
...,...,...
32580,True,10/10 game everyone should buy it
32581,True,best game of the decade. really hope ragnarok ...
32582,True,game fun
32583,True,beautiful
